<a href="https://colab.research.google.com/github/idanye/Applied-CS/blob/working-branch-Idan/Group_14_2024_Applied_Methods_in_CS_Exercise_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1: Waze & Dijkstra's Algorithm

We saw in class how Waze utilizes Dijkstra's Algorithm as a fundamental component of its routing system. Dijkstra's Algorithm is a graph search algorithm that finds the shortest path between two nodes in a weighted graph. In the context of Waze, cities and roads are represented as nodes and edges in a graph, respectively. Each road segment is assigned a weight that reflects the estimated travel time, considering factors such as road type, traffic conditions, and speed limits. When a user inputs a starting and destination point, Waze's implementation of Dijkstra's Algorithm efficiently explores the road network to compute the shortest route. This algorithm ensures that users receive real-time, optimal directions that consider both distance and current traffic conditions.




In [5]:
# All imports
import math
import pandas as pd

# Add more imports here if necessary:
!pip install osmnx
!pip install networkx
!pip install datasets
import osmnx as ox
import networkx as nx
import folium
import heapq
from datasets import load_dataset
import string


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.8/102.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00


# Exercise 1: Waze and other Graphs' Algorithms


<img src="https://pentagram-production.imgix.net/b695e6d0-864c-49f7-8290-ea973d012d0e/LogotypeWazer_Lockups_02-04.jpg?rect=194%2C0%2C3603%2C2251&w=1500&fit=crop&fm=jpg&q=70&auto=format&h=935" width="40%">

Do not start the exercise until you fully understand the submission guidelines, which can be found here.

For any material-related-questions, ask Ami. For any organization-related-questions, ask Lior.


#### **Read the following instructions carefully:**
1. Write your functions in this notebook only. Do not create Python modules and import them.
Feel free to add code blocks if you need.
2. Answers to qualitative questions should be written in markdown cells (with  LATEX  support). Answers that will be written in commented code blocks will not be checked.
3. Kind reminder: the total of all exercises weight is 50% of the course's grade!

#### **This exercise summarizes the following subjects:**
1. Waze (Dijkstra's Algorithm)
2. Autocomplete Algorithm
3. Huffman Coding
4. Markov Chain

## Question 1 (5 points):
1.1 Write a code that reads the `cities.csv` and `roads_matrix.csv` files into memory. You may choose the data structure by yourself.
* `cities.csv` file contains `V=20` cities with columns "City", "Latitude", and "Longitude".
* `roads_matrix.csv` file is a `V*V` matrix, where each cell contains a 0 (no road), a 1 (fast road) or a 2 (slow road).
1.2 Write a function `create_map` that returns a map to visualize cities and roads.


In [ ]:
cities = "cities.csv"
roads_matrix = "roads_matrix.csv"

In [ ]:
def create_map(cities, roads_matrix, show_roads=True):
    """
    Creates a map to visualize cities and roads.
    Hint: You can use the Folium library or any other library of your choice.
    Args:
    - cities (DataFrame): A DataFrame containing city data with columns 'City', 'Latitude', and 'Longitude'.
    - roads_matrix (DataFrame): A DataFrame containing road data between cities.
    - show_roads (bool): Whether to add roads to the map or not.

    Returns:
    - map: The map object.
    """
    # Read the CSV files into pandas DataFrames
    cities_df = pd.read_csv(cities)
    roads_df = pd.read_csv(roads_matrix)

    # Assuming the cities DataFrame has columns 'Latitude' and 'Longitude'
    # Create a list of tuples for cities
    cities = list(zip(cities_df['Latitude'], cities_df['Longitude']))

    # Create a map centered around the first city
    map_center = cities[0] if cities else (0, 0)
    map_obj = folium.Map(location=map_center, zoom_start=5)

    # Add markers for each city
    for idx, (lat, lon) in enumerate(cities):
        folium.Marker([lat, lon], tooltip=f'City {idx}').add_to(map_obj)

    # Add lines for roads if show_roads is True
    # Assuming roads DataFrame has columns 'StartCityIndex' and 'EndCityIndex'
    if show_roads:
        for _, row in roads_df.iterrows():
            start_city = cities[row['StartCityIndex']]
            end_city = cities[row['EndCityIndex']]
            folium.PolyLine([start_city, end_city], color="blue").add_to(map_obj)

    return map_obj

# Test your function:
create_map(cities, roads_matrix, show_roads=True)

## Question 2 (15 points):
Complete the class `RoadNetwork`. This class is responsible for initializing the road network, calculating distances between cities, and estimating travel times on different road types.

Note: You can change the class structure however you see fit, this is just an example to help you get started.

In [ ]:
class RoadNetwork:
    def __init__(self, cities=cities, roads_matrix=roads_matrix):
        pass

    def calculate_distance(self, city1, city2):
        """
        Calculates the distance (in kilometers) between two cities based on their coordinates.

        Returns:
        - distance_km (float): The distance between the two cities in kilometers.
        """
        pass

    def calculate_travel_time(self, source_city, destination_city, road_type):
        """
        Calculates the time (in minutes) it takes to travel from source_city to destination_city, depending on road_type.

        Args:
        - source_city (str): The name of the source city.
        - destination_city (str): The name of the destination city.
        - road_type (int): The type of road (0, 1, or 2).

        Returns:
        - travel_time (float): The travel time in minutes.
        """
        # Parameter p1 tells us how much time does it take to travel in road_type 1 in an avg pace of 25 KM/h
        # Parameter p2 tells us how much time does it take to travel in road_type 2 in an avg pace of 10 KM/h
        p1 = 1 / 25
        p2 = 1 / 10

        # We use parameters r1 and r2 to multiply the L2 distance, to get closer to a real-world road distance between the cities.
        r1 = 2
        r2 = 1.5

        pass

NameError: name 'cities' is not defined

## Question 3 (20 points):
3.1 Write your own Dijkstra's Algorithm to compute the shortest path between two cities.

3.2 Write a function `find_shortest_path` that performs the following tasks:

* Calculates and prints the estimated time of arrival (ETA) for the shortest path.
* Prints the detailed path from the starting city to the destination city.
* Optionally, displays the route on a map object for better visualization.

3.3 Test your function:
*   Find and **visualize** the shortest path from **Ashdod** to **Herzeliya**.
*   Find and **visualize** the shortest path from **Netanya** to **Tel Aviv**.
*   Choose any two cities with a minimum distance of 30 kilometers between them, and find and **visualize** the shortest path between them.



In [ ]:
def dijkstra(graph, start, end):
    """
    Finds the shortest path between two cities using Dijkstra's algorithm.

    Args:
    - graph: The road network object.
    - start: The starting city.
    - end: The destination city.

    Returns:
    - path (list): The shortest path as a list of city names.
    - times (dict): The travel times to each city on the path.
    """
    # Priority queue to hold vertices and their distances from the start
    queue = [(0, start)]
    times = {vertex: float('infinity') for vertex in graph}
    times[start] = 0  # Distance from start to start is obviously 0
    predecessors = {vertex: None for vertex in graph}

    while queue:
        current_distance, current_vertex = heapq.heappop(queue)

        # If we've reached the end, break out of the loop
        if current_vertex == end:
            break

        # If the dequeued distance is greater than the already found shortest distance, skip
        if current_distance > times[current_vertex]:
            continue

        for neighbor, weight in graph[current_vertex].items():
            distance = current_distance + weight

            # If a shorter path to the neighbor is found
            if distance < times[neighbor]:
                times[neighbor] = distance
                predecessors[neighbor] = current_vertex
                heapq.heappush(queue, (distance, neighbor))

    # Reconstruct path from end to start by predecessors
    path = []
    current = end
    while current is not None:
        path.insert(0, current)
        current = predecessors[current]

    return path, times


def find_shortest_path(road_network, start_city, end_city, show_on_map=False):
    """
    Prints the estimated time of arrival (ETA),
    the path from the starting city to the destination,
    and optionally displays the route on a map.

    Args:
    - start_city: The starting city.
    - end_city: The destination city.
    - show_on_map (bool): Whether to display the route on a map.

    Returns:
    - map object or None: The map object if show_on_map is True, otherwise None.
    """
    pass

In [ ]:
# Test your function
# Dummy graph representation
# Graph is represented as a dictionary of dictionaries
# Outer dictionary represents each vertex and its neighbors
# Inner dictionary represents neighbors and their respective distances
dummy_graph = {
    "A": {"B": 1, "C": 4},
    "B": {"A": 1, "C": 2, "D": 5},
    "C": {"A": 4, "B": 2, "D": 1},
    "D": {"B": 5, "C": 1}
}

# Test the function with the dummy graph
path, times = dijkstra(dummy_graph, "A", "D")
path, times

(['A', 'B', 'C', 'D'], {'A': 0, 'B': 1, 'C': 3, 'D': 4})

# Part 2: Autocomplete

In this section, we will explore the autocomplete algorithm. Autocomplete is a feature commonly used in applications, providing word or phrase suggestions as users type. We will be working on implementing this functionality using a data structure called a Trie, as seen on class.

To get started, we will load a dataset that will serve as the basis for your autocomplete system. We will use the Hugging Face `datasets` library. **Hugging Face** is a leading platform for NLP and Machine Learning that provides access to state-of-the-art models, datasets, and tools for developing and deploying applications in the field of natural language understanding. Familiarizing yourself with this platform is valuable if you're interested in NLP and ML.


Before we can load the dataset, we need to install the `datasets` library. The following command will install it: `!pip install datasets -q`

In [6]:
# install the necessary library
!pip install datasets -q

In [7]:
# import and load the dataset
import datasets

## Question 4 (5 points):

Write a function `read_data` to read and preprocess the data.

We'll use a dataset called `google_wellformed_query`. Google's query wellformedness dataset was created by crowdsourcing well-formedness annotations for 25,100 queries from the Paralex corpus. Each query in this dataset has been annotated by five raters, each providing a rating of 1 or 0 to indicate whether or not the query is well-formed. A rating of 1 suggests that the query is well-formed, while a rating of 0 suggests otherwise.

Note: For this assignment, you only need to work with the `train` split of the dataset.

In [8]:
def read_data(dataset, rate):
    """
    This function reads data from a dataset and filters it based on a specified rating threshold.

    Args:
        dataset (dict): The dataset to read from.
        rate (float): The rating threshold. Sentences with a rating greater than or equal to this threshold will be included in the output.

    Returns:
        list: A list of the sentences.
    """
    sentences = []

    for data in dataset:
        if data['rating'] >= rate:
            sentences.append(data['content'])

    return sentences

In [15]:
# Read the data using the dataset library and your function here:
dataset_train = load_dataset("google_wellformed_query", split="train")
# dataset_train
sentences = read_data(dataset_train, 0)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


## Question 5 (15 points):

Complete the Autocomplete function we saw in class, which provides automatic word completions as follows:

*   The function should accept user input as a complete sentence.
*   It will simulate word-by-word printing.
*   Suggestions will be based on probability calculations within a Trie data structure.
*   Probabilities are determined by counters, representing word occurrences in the Trie.

**Example:**

User types **"How do you make the perfect pizza?"**, your function returns the following:
*   **How** many calories in a cup of bluberries
*   **How do** you change the alternator belt on a 1998 audi a4
*   **How do you** make the perfect pizza?
*   **How do you make** the perfect pizza?
*   **How do you make the** perfect pizza?
*   **How do you make the perfect** pizza?
*   **How do you make the perfect pizza?**

**Note:** This is an example based on a specific dataset. Changing or expanding the dataset could potentially yield better results.

**Feel free to make changes and adjustments to the code.**

In [26]:
class TrieNode(object):
    """
    Our trie node implementation. Very basic, but does the job.
      word: The word associated with the node.
      children: A list of child nodes.
      sentence_finished: A boolean indicating if the current node marks the end of a sentence.
      counter: A count of how many times the word associated with this node has appeared in the addition process.
    """
    def __init__(self, word):
        self.word = word
        self.children = {}
        # Is it the last word of the sentence.
        self.sentence_finished = False
        # How many times this word appeared in the addition process
        self.counter = 1

class Trie():
    def __init__(self, text):
        """
        Init a Trie with the given sentences list.
        During initialization, it removes punctuation, converts sentences to lowercase, and adds them to the Trie using the add method.
        """
        self.root = TrieNode('*')
        self.text = text
        self.punctuation = set(string.punctuation)
        for sentence in self.text:
            sentence = ''.join(char for char in sentence if char not in self.punctuation)
            self.add(sentence.lower())

    def add(self, sentence):
        """
        This method adds a sentence to the Trie structure.
        It breaks the sentence into words, and for each word, it traverses the Trie, adding new nodes as necessary or updating existing nodes.
        """
        node = self.root
        for word in sentence.split():
            if word not in node.children:
                new_node = TrieNode(word)
                node.children[word] = new_node
            node = node.children[word]
            node.counter += 1
        node.sentence_finished = True

    def find_prefix(self, prefix):
        """
        Check and return
          1. If the prefix exsists in any of the words we added so far
          2. If yes then how many words actually have the prefix
          3. The node where the prefix ends in the Trie structure
        """
        node = self.root
        # If the root node has no children, then return False.
        # Because it means we are trying to search in an empty trie
        if not self.root.children:
            return False, 0, node

        for word in prefix.split():
            if word in node.children:
                node = node.children[word]
            else:
                return False, 0, node
        return True, node.counter, node

    # def list_all_children(self, prefix):
    #     """
    #     Returns all the children of the node at the end of a prefix, with the counter for each child
    #     """
    #     auto_complete_sentence = prefix
    #     (exists, counter, node) = self.find_prefix(auto_complete_sentence)
    #     print(f'This prefix occurs {counter} times.')

    #     for child in node.children:
    #         print(f"'{child.word}' appears {child.counter} time(s).")

    #     return

    def suggestions_rec(self, node, word):
        """
        Recursive function to collect the end of all words below the given node.
        """
        if node.sentence_finished:
            sentence_list.append((path.strip(), node.counter))

        for child_node in node.children:
            new_path = path + ' ' + child_node.word if path else child_node.word
            self.suggestions_rec(child_node, new_path, sentence_list)

    def auto_complete(self, prefix):
        """
        Suggests the most probable word completion for a given prefix.

        Args:
        - prefix (str): The input prefix to autocomplete.

        Returns:
        - suggestion (str): The suggested word completion.
        """
        exists, _, node = self.find_prefix(prefix)

        if not exists:
            return []

        sentence_list = []
        self.suggestions_rec(node, '', sentence_list)
        sentence_list.sort(key=lambda x: (-x[1], x[0]))

        return [sentence for sentence, _ in sentence_list]


    def init_auto_complete(self):
        """
        Initializes the auto-complete system and allows the user to interactively
        input prefixes and get suggestions.

        """
        prefix = input("Enter a prefix: ")
        suggestions = self.auto_complete(prefix)

        if suggestions:
            print("Suggestions:")

            for suggestion in suggestions:
                print(suggestion)
        else:
            print("No suggestions available for this prefix.")


In [27]:
# Example:
t = Trie(sentences)
t.init_auto_complete()

Enter a prefix: How do you make the perfect pizza?
No suggestions available for this prefix.


# Part 3: Huffman Coding

In these questions, we'll utilize a full English text to create frequency tables for the English language.

We will use a book text to calculate frequencies of all English alphabet letters, including spaces.

Read the book provided through Moodle using the `read_text` function.

In [ ]:
def read_text(file):
    f = open(file, 'r', encoding='utf-8')
    text0 = f.read()
    f.close()

    text0 = text0.lower()

    legalABC = 'abcdefghijklmnopqrstuvwxyz '
    text = ''
    last_was_space = True

    for ch in text0:
        if ch == '\n':
            ch = ' '
        if ch in legalABC:
            if ch != " ":
                text = text + ch
                last_was_space = False
            if ch == " " and last_was_space == False:
                text= text + ch
                last_was_space = True
    return text

text = read_text('book-hw1.txt')

FileNotFoundError: ignored

## Question 6 (10 points):

Create the frequency table and print it.

Count only lowercase letters '**a**' through '**z**' and the **space** character.

**Store frequencies as percentages (like 0.8% or 0.008), not their count.**

Example: {' ': 17, 'a': 8.1, 'b': 1.4, ..., 'z': 0.07 }

In [ ]:
def freq_table(text):
    """
    Calculate the frequency of each character in the text as a percentage of the total characters.
    Sort the results alphabetically.
    Args:
      text (str): your text file.

    Returns:
      freq_table (dict): A dictionary containing characters as keys and their corresponding frequencies as values.
                         The dictionary is sorted alphabetically by character.
    """
    pass

In [ ]:
# print your table here

Following is a table frequency of general English text that includes only the letters '**a**' through '**z**', make sure that your table looks something like this:

Letter | Frequency
--- | :---:
SPACE |
a | 8.167%
b | 1.492%
c | 2.782%
d | 4.253%
e | 12.702%
f | 2.228%
g | 2.015%
h | 6.094%
i | 6.966%
j | 0.253%
k | 1.772%
l | 4.025%
m | 2.406%
n | 6.749%
o | 7.507%
p | 1.929%
q | 0.095%
r | 5.987%
s | 6.327%
t | 9.056%
u | 2.758%
v | 0.978%
w | 2.36%
x | 0.25%
y | 1.974%
z | 0.074%

## Question 7 (10 points):

Write code to construct a Huffman Tree representing English alphabet letters, including spaces.

The goal of this exercise is to create an encoding table that looks something like:

a : 0000

b : 00010

c : 00011

...

**Feel free to make changes and adjustments to the code.**

In [ ]:
class NodeTree(object):
    """
    Represents a node in the Huffman Tree.

    Args:
    left (NodeTree): The left child node (default is None).
    right (NodeTree): The right child node (default is None).

    Methods:
    children(): Returns the left and right children of the node.
    str(): Returns a string representation of the node, including its left and right children.
    """
    def __init__(self, left=None, right=None):
        pass

    def children(self):
        pass

    def __str__(self):
        pass

def huffman_code_tree(node, binString=''):
    """
    Recursively generates Huffman codes for characters in the Huffman Tree.

    Args:
    node (NodeTree): The current node in the Huffman Tree.
    binString (str, optional): The binary string for recursion.

    Returns:
    codes (dict): A dictionary of characters and their corresponding Huffman codes.
    """
    pass

def make_tree(nodes):
    """
    Constructs the Huffman Tree from a list of nodes representing characters and their frequencies.

    Args:
    nodes (list): A list of tuples with character-frequency pairs.

    Returns:
    root (NodeTree): The root node of the Huffman Tree.
    """
    pass

def init_huffman(text, print_code=False):
    """
    Initializes Huffman encoding for a given text.

    Args:
    text (str): The input text to be encoded.
    print_code (bool, optional): Whether to print Huffman codes.

    Returns:
    encoding (dict): A dictionary of characters and their Huffman codes.
    freq (list): A sorted list of tuples representing character frequencies.
    """
    pass

## Question 8 (5 points):
* Encode the first 1000 letters from our text using Huffman coding and print your encoding table.
Determine the number of bits required for Huffman encoding.
Compare this to storing letters in ASCII code, which needs 8 bits per character (totaling 8000 bits for 1000 characters).

* Encode this message in Huffman code: **"The rat and the bear had a date. The rat had a bad hat. The bear had a red beard. Then, the rat had tea and the bear had naan."** Print your encoding table. Determine the number of bits required for Huffman encoding.
Compare this to storing letters in ASCII code, which needs 8 bits per character.

* Answer the open question below.

* **Note:** Don't forget to remove punctuations and use only lowercased letters.

In [ ]:
pass

### Explain in your own words why using Huffman coding is more efficient in the second encoding of the short sentence compared to encoding the entire text.

YOUR ANWER GOES HERE

## Question 9 (5 points):
Here is a list of words:

`Hi, For, Forest, False, Falls, Fall, Friend, Friends, High`

Create binary representations using the Huffman code method.

Build the Huffman Tree for encoding and provide the binary representation for the message "False Friends Fall High."

Use separators between words for clarity.

In [ ]:
pass

# Part 4: Markov Chain

In this exercise, you will work with Markov Chains to generate text based on probabilistic patterns.

## Question 10 (15 points):

1.   **Load and Tokenize:** Begin by loading the `Brown` corpus from the `NLTK` library and tokenize it into words using `word_tokenize`. The `Brown` corpus will serve as our source text.

2.   **Build N-gram Models:** Create various n-gram models (e.g., unigrams, bigrams) to capture word sequences from the tokenized text. While building these models, calculate transition probabilities between n-grams.

3.   **Generate Text:** Implement a text generation function that uses the calculated probabilities. This function should generate text based on the patterns observed in the corpus.

4.   **Print Results:** For each n-gram order (1, 2, 3, and 4-gram), print the generated text. You can inspect the results to understand how different n-gram orders affect the generated text.

**Example of Transition Probabilities:**

For a 2-gram, the probabilities might look like this:

```
{('to', 'wait'): {'one': 0.043478260869565216,
  'until': 0.21739130434782608,
  ',': 0.21739130434782608,
  'for': 0.17391304347826086,
  'a': 0.043478260869565216,
  'his': 0.043478260869565216,
  '.': 0.08695652173913043,
  'to': 0.043478260869565216,
  'till': 0.08695652173913043,
  'before': 0.043478260869565216},...}
```

In [ ]:
import nltk
nltk.download('brown')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import brown

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Set the random seed for 42
SEED = 42
random.seed(SEED)

NameError: ignored

In [ ]:
# Read the text corpus
corpus = None
tokens = None

In [ ]:
def build_ngram_model(tokens, n):
    """
    Build an n-gram model from a list of tokens.

    Args:
    - tokens (list): List of tokens from the corpus.
    - n (int): The order of the n-grams to build (e.g., 1 for unigrams, 2 for bigrams, 3 for trigrams).

    Returns:
    - dict: A dictionary containing n-grams as keys and their associated probability distributions as values.
    """
    ngrams = {}
    pass

In [ ]:
def generate_text(ngrams, length):
    """
    Generate text using an n-gram model.

    Args:
    - ngrams (dict): An n-gram model dictionary generated by build_ngram_model.
    - length (int): The desired length of the generated text in tokens.

    Returns:
    - str: The generated text as a string.
    """
    pass

In [ ]:
# Build n-gram models of different orders with probabilities
n1_gram = build_ngram_model(tokens, n=1)
n2_gram = build_ngram_model(tokens, n=2)
n3_gram = build_ngram_model(tokens, n=3)
n4_gram = build_ngram_model(tokens, n=4)

In [ ]:
# Generate text using the n-gram models with transition probabilities
n1_generated_text = generate_text(n1_gram, length=10)
n2_generated_text = generate_text(n2_gram, length=10)
n3_generated_text = generate_text(n3_gram, length=10)
n4_generated_text = generate_text(n4_gram, length=10)

# Print the generated text
print(f'1-Gram Generated Text: {n1_generated_text}')
print(f'2-Gram Generated Text: {n2_generated_text}')
print(f'3-Gram Generated Text: {n3_generated_text}')
print(f'4-Gram Generated Text: {n4_generated_text}')